# Задание: Оценка тональности по словарю

В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число отзывов, заранее размеченных как положительные или отрицательные; выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

Мы будем работать по заранее определённому пайплайну:

Сначала нам надо скачать данные -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)
Примечание: сбор данных с помощью краулинга может занять много времени, советуем сначала реализовать всё задание на готовых данных, а затем сделать с краулингом, если хотите получить 9 или 10.

In [69]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import time
from pymystem3 import Mystem
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from numpy import array
from pandas.core.common import flatten
import re
from sklearn.model_selection import train_test_split

In [70]:
path_main = r"C:\Users\Abina Kukanova\PycharmProjects\homework"

# Загрузим готовый датасет

Подберем русскоязычный датасет с размеченной тональностью, используем корпус Linis Crowd. 

In [71]:
labeled_texts_1 = pd.read_excel(
    path_main + '\\linis_crowd_dataset\\doc_comment_summary.xlsx',
    sheet_name = 0,
    header=None
    )

In [72]:
labeled_texts_1

,0,1
0,Но при мужчине ни одна приличная женщина не по...,-1
1,Украина это часть Руси искусственно отделенная...,-1
2,Как можно говорить об относительно небольшой к...,-1
3,1.2014. а что они со своими поляками сделали?...,0
4,у а фильмы... Зрители любят диковинное. у ме...,0
...,...,...
26868,Многих заставляют. Многие сами проявляют излиш...,-1
26869,Очередной Чубайс. ну а чего нового то? Сорос ...,-1
26870,"Закон, как все предыдущие - абсолютный бред и ...",0
26871,дело которое ты делаешь сейчас - оно очень хор...,0


In [73]:
labeled_texts_1['label'] = pd.to_numeric(labeled_texts_1.iloc[:, 1], errors='coerce')

In [74]:
labeled_texts_1 = labeled_texts_1[[0, 'label']]

In [75]:
labeled_texts_1.columns = ['text', 'label']
labeled_texts_1.label.value_counts()

label
 0.0        13930
-1.0         9203
 1.0         1795
-2.0         1534
 2.0          365
 22158.0        2
 21887.0        1
 23486.0        1
 23523.0        1
Name: count, dtype: int64

In [76]:
ind_drop = labeled_texts_1.query('label > 2 or label < - 2').index

ind_drop

Index([10224, 15325, 15786, 17474, 26375], dtype='int64')

In [77]:
labeled_texts_1 = labeled_texts_1.query('index not in @ind_drop')

In [78]:
for _ in range(4):

    sample = labeled_texts_1.sample(n = 1)

    print('label: ', sample.label.values[0])

    print(sample['text'].values[0][:200])

    print()

label:  0.0
Будте лояльнее, вам не идут инквизиторские замашки. Я вот жалею что рожала в роддоме и первого и полгода назад второго ребёнка. Я Вас очень прошу: Вы должны досконально изучать материал прежде чем сво

label:  0.0
..Всё самое страшное уже было без России и ничего не только справились , так ещё и поняли что можем и дальше справляться ..  А я знаю человека, котрый считает, что русские в центре России и русские в 

label:  0.0
Но на краю пропасти есть шанс достучаться до здравого смысла в русском человеке - указав ему на пропасть под ногами и заставив вспомнить о предыдущем падении в эту пропасть. В экстремальных ситуациях 

label:  -2.0
сли никогда не штрафуют то хоть 100500.  А они на красный начинают движение? Просто, если на зеленый и потом загорается красный - нужно пропускать, вроде.   Евгений, а почему Вы предлагаете увеличиват



In [79]:
selected = labeled_texts_1.query('label != 0')

In [80]:
selected.label.value_counts()

label
-1.0    9203
 1.0    1795
-2.0    1534
 2.0     365
Name: count, dtype: int64

In [81]:
selected.loc[:, 'label_binary'] = np.nan

C:\Users\Abina Kukanova\AppData\Local\Temp\ipykernel_34376\2569160575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected.loc[:, 'label_binary'] = np.nan


In [82]:
selected.loc[((selected['label'] == -1) |
         (selected['label'] == -2)), 'label_binary'] = 0

selected.loc[((selected['label'] == 1) |
         (selected['label'] == 2)), 'label_binary'] = 1

In [83]:
selected.label_binary.value_counts()

label_binary
0.0    10737
1.0     2160
Name: count, dtype: int64

In [84]:
for _ in range(3):

    sample_neg = selected.query('label_binary == 0').sample(n = 1)

    sample_pos = selected.query('label_binary == 1').sample(n = 1)

    print('label: ', sample_pos.label_binary.values[0])

    print(sample_pos['text'].values[0][:200])

    print('label: ', sample_neg.label_binary.values[0])

    print(sample_neg['text'].values[0][:200])

    print()

label:  1.0
Ответственность за последствия несет тот, кто кинул шланг под ноги прохожим, не оградил законным образом подотчетное имущество, и не предусмотрел его случайное повреждение.В любом случае, если некто в
label:  0.0
я, правда, с собаками когда иду, стараюсь им на глаза не попадаться, они на собак странно смотрят.  это просто праздник какой-то!  Американцы, японцы и корейцы, убейте северян, чтобы они не мучались! 

label:  1.0
отому что самое страшное уже произошло - некоторые Мы родились в материальном мире, которому свойственны ограничения - как то: дорожные знаки, непроходимые насквозь скалы,  репейники, белые линии на д
label:  0.0
от и стараются, как могут!   Избавиться от предпринимателей мечтает не только власть, но и многие оборванцы из числа пролетарствующих маргиналов, которые шипят мне вслед, что я капиталист, буржуй и сп

label:  1.0
ознавательно. Что касается Цзян Цинь - то она оказалась хитрее и понятливее своей предшественницы, и по свидетельству Владимирова, сам

# Лемматизация текстов

In [85]:
#уберем все лишние символы кроме букв, приведем к нижнему регистру
def clear_text(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    return " ".join(clear_text.split())


#удалим стоп-слова
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

In [86]:
# загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

array({'на', 'когда', 'ним', 'в', 'один', 'иногда', 'себе', 'этого', 'то', 'через', 'к', 'два', 'вас', 'этот', 'наконец', 'ничего', 'все', 'да', 'там', 'более', 'раз', 'перед', 'были', 'быть', 'про', 'потому', 'хоть', 'совсем', 'что', 'надо', 'тебя', 'если', 'ж', 'бы', 'еще', 'даже', 'они', 'ему', 'тот', 'тогда', 'как', 'у', 'меня', 'до', 'чтобы', 'всего', 'для', 'был', 'тоже', 'моя', 'будет', 'мой', 'потом', 'мы', 'другой', 'ли', 'него', 'была', 'всех', 'нее', 'можно', 'вдруг', 'между', 'я', 'с', 'такой', 'его', 'она', 'какая', 'же', 'за', 'он', 'при', 'разве', 'опять', 'мне', 'нет', 'о', 'где', 'или', 'без', 'по', 'ни', 'их', 'тут', 'ней', 'эти', 'всю', 'во', 'свою', 'будто', 'может', 'над', 'но', 'этой', 'чем', 'впрочем', 'нибудь', 'от', 'кто', 'из', 'них', 'чего', 'здесь', 'нас', 'вам', 'себя', 'тем', 'лучше', 'сейчас', 'им', 'после', 'а', 'много', 'хорошо', 'есть', 'куда', 'чтоб', 'так', 'под', 'нельзя', 'три', 'ведь', 'теперь', 'больше', 'со', 'уже', 'эту', 'том', 'сам', 'зачем',

In [87]:
start_clean = time.time()

selected['text_clear'] = selected['text'].apply(lambda x:clean_stop_words(clear_text(str(x)),stopwords))

print('Обработка текстов заняла: '+str(round(time.time() - start_clean, 2))+' секунд')

Обработка текстов заняла: 0.97 секунд


C:\Users\Abina Kukanova\AppData\Local\Temp\ipykernel_34376\1966760752.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['text_clear'] = selected['text'].apply(lambda x:clean_stop_words(clear_text(str(x)),stopwords))


In [88]:
def lemmatize(df : (pd.Series, pd.DataFrame),
              text_column : (None, str),
              n_samples : int,
              break_str = 'br',
             ) -> pd.Series:


    result = []

    m = Mystem()

    for i in tqdm(range((df.shape[0] // n_samples) + 1)) :

        start = i * n_samples

        stop = start + n_samples

        sample = break_str.join(df[text_column][start : stop].values)

        lemmas = m.lemmatize(sample)

        lemm_sample = ''.join(lemmas).split(break_str)

        result += lemm_sample

    return pd.Series(result, index = df.index)

In [89]:
selected['lemm_clean_tex'] = lemmatize(
    df = selected,
    text_column = 'text_clear',
    n_samples = 100,
    break_str = 'br',
    )

  0%|          | 0/130 [00:00<?, ?it/s]

100%|██████████| 130/130 [01:59<00:00,  1.08it/s]
C:\Users\Abina Kukanova\AppData\Local\Temp\ipykernel_34376\959808114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['lemm_clean_tex'] = lemmatize(


In [90]:
selected.head()

,text,label,label_binary,text_clear,lemm_clean_tex
0,Но при мужчине ни одна приличная женщина не по...,-1.0,0.0,мужчине одна приличная женщина пойдет лазить р...,мужчина один приличный женщина пойти лазить ра...
1,Украина это часть Руси искусственно отделенная...,-1.0,0.0,украина это часть руси искусственно отделенная...,украина это часть русь искусственно отделять к...
2,Как можно говорить об относительно небольшой к...,-1.0,0.0,говорить относительно небольшой коррупции обра...,говорить относительно небольшой коррупция обра...
5,Государство не может сейчас платить больше и м...,-1.0,0.0,государство платить посмотрите денежный поток ...,государство платить посмотреть денежный поток ...
8,эксплуатируемые способны только на бунты - бес...,-1.0,0.0,эксплуатируемые способны бунты бессмысленные б...,эксплуатируемые способный бунт бессмысленный б...


In [91]:
selected['lemm_clean_tex']


0        мужчина один приличный женщина пойти лазить ра...
1        украина это часть русь искусственно отделять к...
2        говорить относительно небольшой коррупция обра...
5        государство платить посмотреть денежный поток ...
8        эксплуатируемые способный бунт бессмысленный б...
                               ...                        
26864    великое испытание выпадать доля наш предок веч...
26865    н свой поведение крайне низкий человек прощать...
26866    многие психологический книга кричать женщина д...
26868    многих заставлять многий сам проявлять излишни...
26869    очередной чубайс новый сорос х давать исчерпыв...
Name: lemm_clean_tex, Length: 12938, dtype: object

In [92]:
#разделим на положительные и отрицательные
positive_df = selected[selected['label_binary'] == 1.0]
negative_df = selected[selected['label_binary'] == 0.0]

In [93]:
positive_tokens = []
for line in positive_df['lemm_clean_tex']:
    positive_tokens.append(word_tokenize(line))

negative_tokens = []
for line in negative_df['lemm_clean_tex']:
    negative_tokens.append(word_tokenize(line))

In [94]:
import operator
from functools import reduce
positive_tokens = reduce(operator.concat, positive_tokens)
negative_tokens = reduce(operator.concat, negative_tokens)

In [95]:
#Возвращает словарь {слово: абсолютная частота встречаемости} на основе списка слов positive_tokens
pos_freq = {}
for pos_word in positive_tokens:
    if pos_word in pos_freq:
        pos_freq[pos_word] += 1
    else:
        pos_freq[pos_word] = 1

In [96]:
#Возвращает словарь {слово: абсолютная частота встречаемости} на основе списка слов negative_tokens
neg_freq = {}
for neg_word in negative_tokens:
    if neg_word in neg_freq:
        neg_freq[neg_word] += 1
    else:
        neg_freq[neg_word] = 1

In [97]:
#выбросим слова, которые встречаются только 1-2 раза
pos_words = []
for pos_word, freq_word in pos_freq.items():
    if freq_word > 2:
        pos_words.append(pos_word)

neg_words = []
for neg_word, freq_word in neg_freq.items():
    if freq_word > 2:
        neg_words.append(neg_word)

In [98]:
pos_set = set(pos_words)
neg_set = set(neg_words)

У нас есть два множества: положительные pos_set и негативные neg_set. Разница между pos_set и neg_set (pos_set — neg_set) — это множество со всеми элементами, которые содержатся в pos_set, но не в neg_set. Соответственно, (neg_set — pos_set) — это множество со всеми элементами в neg_set, но не в pos_set.

In [99]:
diff_pos = pos_set.difference(neg_set)
diff_neg = neg_set.difference(pos_set)

В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число отзывов, заранее размеченных как положительные или отрицательные; выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

In [101]:
m = Mystem()
def sentiment_analysis(reply):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    lemmas = m.lemmatize(clear_text)
    lemm_text = ''.join(lemmas)
    tokens = word_tokenize(lemm_text)
    sample_set = set(tokens)
    if sample_set.intersection(diff_pos) > sample_set.intersection(diff_neg):
        print('positive')
    else:
        print('negative')

In [102]:
#В нашем датасете этот комментарий был негативным, и наша функция вывела, что этот комментарий тоже негативный
text = """html  Еще пара таких инициатив и я тоже буду голосовать за Мерец.
  Ну чо, 20 лет прошло поколение сменилось , можно опять, как в 92 -м ,
    сделать кучу мандатов на борьбе с религиозным засильем и за соц"""
b = sentiment_analysis(text)
b

negative


In [103]:
#вытащим первые пять положительных и отрицательных комментариев, чтобы проверить функцию и рассчитать accuracy
scrapped = {}
for line in positive_df['text'].head():
    scrapped[line] = "positive"

for line in negative_df['text'].head():
    scrapped[line] = "negative"

In [104]:
len(scrapped)

10

In [105]:
y_true = list(scrapped.values())
scrapped_keys = list(scrapped.keys())

In [106]:
m = Mystem()
b = []
def sentiment(reply):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    lemmas = m.lemmatize(clear_text)
    lemm_text = ''.join(lemmas)
    tokens = word_tokenize(lemm_text)
    sample_set = set(tokens)
    if sample_set.intersection(diff_pos) > sample_set.intersection(diff_neg):
        b.append('positive')
    else:
        b.append('negative')
    return b

In [107]:
for key in scrapped_keys:
    y_pred = sentiment(key)

In [108]:
len(y_pred)

10

In [109]:
y_pred

['negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative']

In [110]:
import numpy
import sklearn.metrics


r = sklearn.metrics.confusion_matrix(y_true, y_pred)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.5


На вход нашей функции мы подали 10 комментариев, первые пять из которых были положительными, а другие отрицательными, но наша функция вывела, что все комментарии отрицательные. На основе матрицы ошибок рассчитали accuracy и получили 0,5. Вы можете видеть, что точность довольно низкая – всего 50%, что довольно разочаровывает при всей этой работе.

Предложите как минимум 2 способа улучшить этот алгоритм определения тональности отзыва (1 балл за описание и реализацию каждого способа; если 2 способа описаны только текстом, это 1 балл. За третий и последующие способы дополнительных баллов не будет)

# Первый способ

Мы возьмем только существительные, глаголы, прилагательные и наречия из наших комментариев. Мы будем использовать SentiWordNet для вычисления полярности. Тональностью будет разница между положительным и отрицательным результатом. Возьмем список синонимов и и первый синоним в этом списке для вычисления отношения . Тональность каждого комментария будет средним значением каждого слова. 

In [111]:
selected = selected.dropna()

In [112]:
#разделим данные на обучающую и тестовую выборки 
train_X, test_X, train_y, test_y = train_test_split(selected['text'], selected['label_binary'], test_size=0.2, random_state=0)

In [113]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Abina Kukanova\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [114]:
import nltk
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to C:\Users\Abina
[nltk_data]     Kukanova\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [115]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [116]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

def sentiment_sentiwordnet(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', str(text)).lower()
    raw_sentences = sent_tokenize(clear_text)
    sentiment = 0
    tokens_count = 0

    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            word_sent = swn_synset.pos_score() - swn_synset.neg_score()

            if word_sent != 0:
                sentiment += word_sent
                tokens_count += 1

    if tokens_count == 0:
        return 0
    sentiment = sentiment/tokens_count
    if sentiment >= 0.01:
        return 1
    if sentiment <= -0.01:
        return -1
    return 0

In [117]:
from sklearn.metrics import accuracy_score
pred_y = [sentiment_sentiwordnet(text) for text in test_X]
accuracy_score(test_y.to_list(), pred_y)

0.8483908491663436

Видим, что accuracy значительно увеличилась - 0.8483908491663436. 

# Второй способ 

Используем модель логистической регрессии, чтобы предсказать положительные и отрицательные отзывы. 

In [118]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [119]:
train_X, test_X, train_y, test_y = train_test_split(selected['text'], selected['label_binary'], test_size=0.2, random_state=0)

In [120]:
from nltk.corpus import stopwords
stops = stopwords.words("russian")
tfidf = TfidfVectorizer(stop_words=stops, binary=True, max_features=10000)
tfidf.fit(train_X)
X_train = tfidf.transform(train_X)
X_test = tfidf.transform(test_X)

In [121]:
model = LogisticRegression()
model.fit(X_train, train_y)
y_pred = model.predict(X_test)
accuracy_score(test_y, y_pred)

0.879798371461807

Использовав  модель логистической регрессии, мы получили accuracy больше, чем в нашем первом алгоритме и в первом способе - 0.879798371461807. 